# Bank Churning Using Deep Learning
Dataset Link: https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling

Churn = customers lost over a period of time
For detailed data exploring, check this article: https://towardsdatascience.com/churn-prediction-with-machine-learning-ca955d52bd8c

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
df = pd.read_csv("/content/Churn_Modelling.csv")
df.sample(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
8745,8746,15793424,Tan,663,Spain,Female,28,8,61274.70,2,1,0,136054.45,0
4974,4975,15744942,Steele,638,Spain,Female,55,2,155828.22,1,0,1,108987.25,1
2165,2166,15585041,Ainsworth,511,France,Male,33,7,0.00,2,0,1,158313.87,0
5405,5406,15592707,Dolgorukova,531,Germany,Female,64,2,175754.87,2,1,1,60721.40,0
773,774,15685320,Johnstone,767,France,Male,36,3,139180.20,1,0,0,123880.19,0


In [3]:
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

In [4]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [5]:
# Creating a function to iterate through columns that consist of logical values:
def print_unique_col_values(df):
  for column in df:
    if df[column].dtypes=='object':
      print(f'{column}: {df[column].unique()}') 

In [6]:
print_unique_col_values(df)

Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


In [7]:
df['Gender'].replace({'Male':0,'Female':1},inplace=True)

In [8]:
for col in df:
  print(f'{col}: {df[col].unique()}')

CreditScore: [619 608 502 699 850 645 822 376 501 684 528 497 476 549 635 616 653 587
 726 732 636 510 669 846 577 756 571 574 411 591 533 553 520 722 475 490
 804 582 472 465 556 834 660 776 829 637 550 698 585 788 655 601 656 725
 511 614 742 687 555 603 751 581 735 661 675 738 813 657 604 519 664 678
 757 416 665 777 543 506 493 652 750 729 646 647 808 524 769 730 515 773
 814 710 413 623 670 622 785 605 479 685 538 562 721 628 668 828 674 625
 432 770 758 795 686 789 589 461 584 579 663 682 793 691 485 650 754 535
 716 539 706 586 631 717 800 683 704 615 667 484 480 578 512 606 597 778
 514 525 715 580 807 521 759 516 711 618 643 671 689 620 676 572 695 592
 567 694 547 594 673 610 767 763 712 703 662 659 523 772 545 634 739 771
 681 544 696 766 727 693 557 531 498 651 791 733 811 707 714 782 775 799
 602 744 588 747 583 627 731 629 438 642 806 474 559 429 680 749 734 644
 626 649 805 718 840 630 654 762 568 613 522 737 648 443 640 540 460 593
 801 611 802 745 483 690 492 709 705 5

In [9]:
#One hot encoding other columns
df1 = pd.get_dummies(data=df, columns=['Geography'])
df1.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [10]:
df1.dtypes

CreditScore            int64
Gender                 int64
Age                    int64
Tenure                 int64
Balance              float64
NumOfProducts          int64
HasCrCard              int64
IsActiveMember         int64
EstimatedSalary      float64
Exited                 int64
Geography_France       uint8
Geography_Germany      uint8
Geography_Spain        uint8
dtype: object

In [11]:
#In Deep Learning, it is imp to scale the data to ensure that it is between some range
df1.sample(5)


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
9687,791,1,31,10,75499.24,1,1,0,22184.14,0,1,0,0
737,525,0,36,2,114628.40,1,0,1,168290.06,0,1,0,0
1737,769,0,39,9,0.00,1,1,1,47722.79,0,0,0,1
5470,617,0,25,1,102585.88,2,1,1,115387.40,0,1,0,0
5454,704,0,39,5,0.00,1,1,0,6416.92,0,1,0,0


In [12]:
cols_to_scale = ['CreditScore','Balance','EstimatedSalary', 'Age', 'Tenure', 'NumOfProducts']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df1[cols_to_scale] = scaler.fit_transform(df1[cols_to_scale])

In [13]:
df1.sample(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
9283,0.424,1,0.121622,0.3,0.000000,0.333333,1,0,0.140641,0,1,0,0
4104,1.000,0,0.500000,0.6,0.000000,0.000000,1,0,0.004665,1,1,0,0
574,0.996,0,0.297297,0.3,0.442132,0.000000,1,1,0.154341,0,0,0,1
4085,0.340,0,0.216216,0.4,0.534113,0.000000,1,1,0.966080,0,1,0,0
5298,0.690,0,0.243243,0.6,0.454398,0.333333,1,0,0.590603,0,1,0,0


In [14]:
#train test split for our ANN
X = df1.drop('Exited',axis='columns')
y = df1['Exited']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=5)

In [15]:
X_train.shape

(8000, 12)

In [16]:
X_test.shape

(2000, 12)

In [17]:
X_train[:10]

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
7751,0.800,1,0.283784,0.6,0.000000,0.333333,0,0,0.096273,0,0,1
4154,0.752,0,0.216216,0.3,0.000000,0.333333,1,0,0.981478,1,0,0
3881,0.476,1,0.621622,0.3,0.000000,0.000000,1,1,0.948551,1,0,0
9238,0.846,1,0.432432,0.4,0.000000,0.333333,1,0,0.646869,1,0,0
5210,0.402,0,0.229730,0.7,0.517012,0.333333,0,0,0.434670,1,0,0
7487,0.602,1,0.513514,0.4,0.000000,0.000000,0,0,0.421898,1,0,0
7542,0.314,1,0.216216,0.4,0.000000,0.333333,1,1,0.303413,0,0,1
7524,0.620,1,0.297297,0.8,0.666330,0.000000,1,1,0.925815,1,0,0
9412,0.750,0,0.108108,0.6,0.393324,0.000000,0,0,0.668609,1,0,0
6377,0.684,0,0.202703,0.9,0.000000,0.000000,1,0,0.567526,1,0,0


In [18]:
len(X_train.columns)

12

In [19]:
import tensorflow as tf
from tensorflow import keras


#model = keras.Sequential([
   # keras.layers.Dense(26, input_shape=(26,)), #input layer
  #  keras.layers.Dense(15, activation='relu'), #hidden layer
 #   keras.layers.Dense(1, activation='sigmoid') #ouput layer
#])

#or this will also work
model = keras.Sequential([
    keras.layers.Dense(12, input_shape=(12,), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid') #ouput layer
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50)

Epoch 1/50
250/250 [==============================] - 4s 2ms/step - loss: 0.5501 - accuracy: 0.7620
Epoch 2/50
250/250 [==============================] - 1s 2ms/step - loss: 0.4825 - accuracy: 0.7960
Epoch 3/50
250/250 [==============================] - 0s 2ms/step - loss: 0.4702 - accuracy: 0.7960
Epoch 4/50
250/250 [==============================] - 1s 2ms/step - loss: 0.4631 - accuracy: 0.7968
Epoch 5/50
250/250 [==============================] - 1s 2ms/step - loss: 0.4577 - accuracy: 0.7977
Epoch 6/50
250/250 [==============================] - 1s 2ms/step - loss: 0.4527 - accuracy: 0.7975
Epoch 7/50
250/250 [==============================] - 0s 2ms/step - loss: 0.4480 - accuracy: 0.7996
Epoch 8/50
250/250 [==============================] - 1s 2ms/step - loss: 0.4434 - accuracy: 0.8030
Epoch 9/50
250/250 [==============================] - 1s 2ms/step - loss: 0.4391 - accuracy: 0.8035
Epoch 10/50
250/250 [==============================] - 0s 2ms/step - loss: 0.4353 - accuracy: 0.8073

In [20]:
yp = model.predict(X_test)

In [21]:
#Checking the first few values of prediction
yp[:5]

array([[0.05736319],
       [0.07952155],
       [0.08125153],
       [0.08114218],
       [0.09627903]], dtype=float32)

In [22]:
#Convert probabilistic values to 0,1
y_pred = []
for element in yp:
  if element > 0.5:
    y_pred.append(1)
  else:
    y_pred.append(0)

In [23]:
y_pred[:5]

[0, 0, 0, 0, 0]

In [24]:
#Evaluating the prediction
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.97      0.91      1595
           1       0.76      0.42      0.54       405

    accuracy                           0.86      2000
   macro avg       0.81      0.69      0.73      2000
weighted avg       0.85      0.86      0.84      2000

